In [51]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
import re

In [72]:
# 페이지 크롤링 함수
def cardCrawling (cardurl):
    benefits=""
    benefit_summary=""
    
    cardhtml=urlopen(cardurl)
    cardbs=BeautifulSoup(cardhtml,'html.parser')
    moa=0

    # benefit_summary
    try:
        summaryList=cardbs.find('div',{'class','bnfCont smry'})
        summaries=summaryList.findAll('li')
        for summary in summaries:
            sumname=summary.find('b',{'class','tit'}).text
            if sumname=="L.POINT" or sumname=="가족카드" or sumname== "가족카드 안내" or sumname== "연회비": continue
            sumcontent=summary.find('span',{'class','svTit'}).text
            benefit_summary+="["+sumname+"] "+sumcontent+"\n"
    except:
        print("혜택 모아보기 없음")
        moa=1
    
    # benefits
    benes = cardbs.findAll('div',{'class','bnfCont'})
    if len(benes) == 0:
        beneLists = cardbs.find('ul',{'class','toggleList'}).findAll('li',recursive=False)
        for beneList in beneLists:
            titlebene = beneList.find('a').text.replace('\t','').replace('\r','').replace('\n','').replace(' ','')
            if titlebene=="L.POINT" or titlebene=="가족카드" or titlebene== "가족카드안내" or titlebene== "연회비": continue
            benefits+="<"+titlebene+"> "
            details = beneList.find('div',{'class','toggleCont'}).findAll(recursive=False)
            for detail in details:
                try:
                    thead = detail.find('thead').findAll('th')
                    sentence = "표의 칼럼은 "

                    for columnname in thead:
                        columnname=columnname.text
                        sentence+=columnname.replace('\n','')+", "
                    sentence=sentence[0:-2]
                    sentence += " 로 이루어져 있습니다."
                    benefits+=sentence
                    j=1
                    tbodies = detail.find('tbody').findAll('tr')

                    for tbody in tbodies:
                        tds = tbody.findAll('td')
                        sentence += f"표의 {j}번째 행은 "
                        for td in tds:
                            sentence+=td.text.replace(",","").replace('\n','')+", "
                        sentence=sentence[0:-2]
                        sentence += " 로 이루어져 있습니다."
                        j+=1
                    benefits+=sentence 

                except:
                    if str(detail)[1:3]=="h3": 
                        benefits+="["+detail.text+"]"
                    elif str(detail)[1:3]=="h4": 
                        benefits+="/"+detail.text+": "
                    elif str(detail)[1:6]=="style": 
                        continue
                    else:
                        benefits+=detail.text.replace("\n","").replace('\r','').replace('\t','')
                benefits.replace('\n','').replace('\t','')
            benefits+="\n"
    else:
        if moa==0: benes=benes[1:]
        for bene in benes:
            
            titlebene=bene.find('h3').text
            if titlebene=="L.POINT" or titlebene=="가족카드" or titlebene== "가족카드 안내" or titlebene== "연회비": continue
            benefits+="<"+titlebene+">"
            sections = bene.findAll('div',{'class','toggle'})
            for section in sections:
                beneNames=section.find('h4')
                beneNameText=beneNames.text
                benefits+="["+beneNameText+"] "
                details = section.find('div',{'class','toggleCont'}).findAll(recursive=False)

                for detail in details:
                    try:
                        thead = detail.find('thead').findAll('th')
                        sentence = "표의 칼럼은 "
                        for columnname in columnnames:
                            columnname=columnname.text
                            sentence+=columnname.replace('\n','')+", "
                        sentence=sentence[0:-2]
                        sentence += " 로 이루어져 있습니다."
                        benefits+=sentence
                        j=1
                        tbodies = detail.find('tbody').findAll('tr')
                        for tbody in tbodies:
                            tds = tbody.findAll('td')
                            sentence += f"표의 {j}번째 행은 "
                            for td in tds:
                                sentence+=td.text.replace(",","").replace('\n','')+", "
                            sentence=sentence[0:-2]
                            sentence += " 로 이루어져 있습니다."
                            j+=1
                        benefits+=sentence 

                    except:
                        if str(detail)[1:3]=="h5": 
                            benefits+="/"+detail.text.replace("\n","").replace('\r','').replace('\t','')+": "
                        elif str(detail)[1:7]=="script": 
                            continue
                        else:
                            benefits+=detail.text.replace("\n","").replace('\r','').replace('\t','')
            benefits+="\n"

    
    return benefits, benefit_summary 

In [73]:
# 신한카드

In [74]:
url = 'https://www.lottecard.co.kr/app/LPCDADA_V100.lc'

chrome_options = Options()
chrome_options.add_argument("--headless")
driver = webdriver.Chrome(options=chrome_options)

# 웹 페이지 로드
driver.get(url)

# Selenium으로 페이지 스크랩
rendered_html = driver.page_source

# BeautifulSoup을 사용하여 HTML 파싱
soup = BeautifulSoup(rendered_html, 'html.parser')

In [75]:
# 더보기 버튼 끝까지 누르기
while True:
    new_render_html=driver.page_source
    soup = BeautifulSoup(new_render_html, 'html.parser')
    if soup.find('button',{'id':'btnMore'}) is None: break
    driver.find_element("id","btnMore").click()
    time.sleep(3)

In [ ]:
cards=soup.find('ul', {'id':'ajaxCardList'}).findAll('li')
for card in cards:
    cardNo = card.find('a').get('onclick')
    cardNo = re.search(r"'(.*?)'", cardNo).group(1)
    print(cardNo)
    cardurl='https://www.lottecard.co.kr/app/LPCDADB_V100.lc?vtCdKndC='+cardNo
    print(cardurl)
    
    cardImg= "https:" + card.find('img').get('src')
    print(cardImg)
    
    cardName=card.find('b').text
    print(cardName)
    
    benefits,benefit_summary = cardCrawling(cardurl)
    print(benefit_summary)
    print(benefits)   
    
    print()
    print()

P14312-A14312
https://www.lottecard.co.kr/app/LPCDADB_V100.lc?vtCdKndC=P14312-A14312
https://image.lottecard.co.kr/UploadFiles/ecenterPath/cdInfo/ecenterCdInfoP14312-A14312_nm1_v.png
롯데마트&MAXX 카드
[롯데마트] 롯데마트, MAXX 월 최대 30,000원 결제일 할인
[생활업종] 5대 생활업종 월 최대 25,000원 결제일 할인
[서비스 제공기준] 롯데마트&MAXX 카드 할인 공통기준

<롯데마트>[롯데마트, MAXX 월 최대 30,000원 결제일 할인] 지난달 이용실적, 이번 달 할인율, 이번 달 할인한도로 구성되어 있습니다.지난달 이용실적이번 달 할인율이번 달 할인한도50만원 미만1천원 미만 금액 할인일 1회, 월 5회 (최대 4,995원)50만원 이상 100만원 미만5%일 5,000원 월 15,000원100만원 이상10%일 10,000원 월 30,000원롯데마트&MAXX 카드를 최초 발급하신 경우 카드발급일~다음 달 말일까지 이용실적에 관계없이 5% 할인(일 5,000원 / 월 15,000원 한도) 혜택이 제공됩니다.카드발급일~다음 달 말일 기간에도 지난달(1일~말일) 롯데마트&MAXX 카드 이용실적이 100만원 이상일 경우, 해당 이용실적에 따라 할인 혜택이 제공됩니다.전국 롯데마트, MAXX(VIC마켓 포함) 오프라인 매장 및 온라인 롯데마트몰(www.lottemart.com) 이용건만 할인 혜택이 제공됩니다.
<생활업종>[5대 생활업종 월 최대 25,000원 결제일 할인] /병원·약국·동물병원, 주유, 교육, 이동통신, 스트리밍: 지난달 이용실적, 이번 달 할인율, 이번 달 할인한도로 구성되어 있습니다.지난달 이용실적이번 달 할인율이번 달 할인한도50만원 이상 100만원 미만5%월 15,000원 (업종별 3,000원)100만원 이상월 25,000원 (업종별 5,000원)지난달 1일~말일까지 50만원 이

[모든 가맹점] 모든 가맹점 할인
[온라인] 온라인 할인
[공통 안내] 할인 공통기준

<모든 가맹점>[모든 가맹점 할인] /어디서나 1.2% 결제일 할인: 국내 모든 가맹점 1.2%, 해외 모든 이용금액 1.2%실적조건 및 할인한도 없이 혜택을 제공합니다.
<온라인>[온라인 할인] /온라인 1.5% 결제일 할인: 실적조건 및 할인한도 없이 혜택을 제공합니다.
<공통 안내>[할인 공통기준] 롯데카드 가맹점 및 업종 분류 기준으로 혜택을 제공합니다.모든 가맹점 할인과 온라인 할인은 중복으로 적용되지 않으며, 할인이 큰 혜택으로 적용됩니다.해외 이용 시 별도의 수수료가 부과되오니, 상품설명서 뒷면을 확인해주세요.할인 적용 제외 대상국세, 지방세, 도시가스비, 공과금, 건강보험, 국민연금, 고용보험, 산재보험, 초·중·고교 납입금, 대학등록금, LH·SH 공사 공공임대료, 부동산 임대료, 장애인 고용부담금, 아파트관리비, 오토·스마트 캐시백 신청 이용금액, 기프트·선불카드 충전 및 구매, 포인트 충전, 상품권(모바일 상품권 포함) 구매, 대중교통(버스,지하철), 택시, 고속도로 통행요금, 고속버스, 무승인전표(자판기, 터널통행료, 항공 기내 이용), 벌금, 과태료, 민원 발급 수수료, 관세 납부, 인지세, 송달료, 모든 무이자할부 이용금액, 단기카드대출(현금서비스), 장기카드대출(카드론), 연회비, 이자 및 각종 수수료[이용기준] 롯데카드의 다른 결제일 할인(청구할인) 혜택과 중복 시 할인이 큰 혜택으로 제공됩니다. (중복 혜택 적용 불가)롯데카드 가맹점 및 업종 분류 기준으로 혜택이 제공됩니다.할부 이용 시 이용일 기준 전체 할부 이용금액에 대하여 혜택이 제공됩니다.이동통신 자동납부 건은 한 달동안 이용한 금액이 다음 달에 결제되며, 롯데카드에 접수되는 일자를 기준으로 혜택이 제공됩니다.해외 이용금액은 롯데카드에 접수되는 현지 매출일자를 기준으로 혜택이 제공되며, 롯데카드에 접수된 후 확인 가능합니다.결제 취소 건 접수 내역은 롯데카드 홈페이지(www.lottec

[주유] SK에너지, S-OIL, GS칼텍스, 현대오일뱅크 60% 결제일 할인
[영화] 롯데시네마, CGV, 메가박스 60% 결제일 할인
[스트리밍] 넷플릭스, 왓챠, 유튜브, 멜론, 지니 60% 결제일 할인
[멤버십] 쿠팡 로켓와우, 네이버플러스 멤버십 60% 결제일 할인
[공통안내] 할인 공통기준

<주유>[SK에너지, S-OIL, GS칼텍스, 현대오일뱅크 60% 결제일 할인] LPG 충전소 이용금액은 혜택이 제공되지 않습니다.
<영화>[롯데시네마, CGV, 메가박스 60% 결제일 할인] 현장 결제, 공식 홈페이지 및 앱을 통한 예매 시 할인 혜택이 제공되며, 기프티콘, 상품권 구매 등 영화 예매 이외의 건은 혜택이 제공되지 않습니다.
<스트리밍>[넷플릭스, 왓챠, 유튜브, 멜론, 지니 60% 결제일 할인] 공식 홈페이지 및 앱을 통해 직접 신청한 정기결제(자동납부) 이용 건만 혜택이 제공되며, 1회성 일반구매 결제 및 통신요금 내 합산 청구, 앱마켓(구글플레이·앱스토어 등) 결제와 같이 가맹점 구분이 불가한 경우에는 혜택이 제공되지 않습니다.
<멤버십>[쿠팡 로켓와우, 네이버플러스 멤버십 60% 결제일 할인] /쿠팡 로켓와우, 네이버플러스 멤버십 60% 결제일 할인: 
<공통안내>[할인 공통기준] 지난달 1일~말일까지 LOCA LIKIT Play 카드로 40만원 이상 이용 시 혜택이 제공됩니다.할인한도는 모든 혜택 통합 월 1만 3천원 한도 내에서 제공합니다.롯데카드 가맹점 및 업종 분류 기준으로 혜택을 제공합니다.할인 적용 제외 대상모든 무이자할부 이용금액, 포인트 충전, 기프트·선불카드 충전 및 구매, 상품권(모바일 상품권 포함) 구매[실적기준] LOCA LIKIT Play 카드를 최초 발급받은 경우 카드발급일~다음 달 말일까지 이용실적에 관계없이 혜택이 제공됩니다.지난달 실적 산정 시 가족카드 이용금액도 동일한 기준으로 본인카드 이용실적에 포함됩니다.취소 매출은 실제 승인된 달의 실적에서 제외하며, 실적에 포함된 매출을  취소하는 경우 실적 금액 및

In [ ]:
# 체크 카드

In [ ]:
url = 'https://www.lottecard.co.kr/app/LPCDAEA_V100.lc'

chrome_options = Options()
chrome_options.add_argument("--headless")
driver = webdriver.Chrome(options=chrome_options)

# 웹 페이지 로드
driver.get(url)

# Selenium으로 페이지 스크랩
rendered_html = driver.page_source

# BeautifulSoup을 사용하여 HTML 파싱
soup = BeautifulSoup(rendered_html, 'html.parser')

In [ ]:
# 더보기 버튼 끝까지 누르기
while True:
    new_render_html=driver.page_source
    soup = BeautifulSoup(new_render_html, 'html.parser')
    if soup.find('button',{'id':'btnMore'}) is None: break
    driver.find_element("id","btnMore").click()
    time.sleep(3)

In [ ]:
cards=soup.find('ul', {'id':'ajaxCardList'}).findAll('li')
for card in cards:
    cardNo = card.find('a').get('onclick')
    cardNo = re.search(r"'(.*?)'", cardNo).group(1)
    print(cardNo)
    cardurl='https://www.lottecard.co.kr/app/LPCDADB_V100.lc?vtCdKndC='+cardNo
    print(cardurl)
    
    cardImg= "https:" + card.find('img').get('src')
    print(cardImg)
    
    cardName=card.find('b').text
    print(cardName)
    
    benefits,benefit_summary = cardCrawling(cardurl)
    print(benefit_summary)
    print(benefits)   
    
    print()
    print()